# 1. Import libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import roc_auc_score, confusion_matrix

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
def IV (data, X, Y):    
    datag = data[[X,Y]]
    datag = datag.groupby(X)[Y].agg(['sum','count'])
    datag['bad'] = datag['sum']
    datag['good'] = datag['count'] - datag['sum']
    datag['pct_bad'] = datag['bad'] / datag['bad'].sum()
    datag['pct_good'] = datag['good'] / datag['good'].sum()
    datag['WOE'] = np.log(datag['pct_bad'] / datag['pct_good'])
    datag['IV'] = datag['WOE'] * (datag['pct_bad'] - datag['pct_good'])
    print(X,': ',round(datag['IV'].sum(),3))

In [4]:
def VIF (data):
    """number of arguments = 1
    arg 1 (data): enter the dataset name"""
    X = add_constant(data)
    Y = pd.Series([variance_inflation_factor(X.values, i)
                   for i in range(X.shape[1])],
                  index=X.columns)
    print(Y)

# 2. Import datasets

In [5]:
df = pd.read_csv("../inputs/1_Bins.csv")
df.shape

(9578, 13)

# 3. IV

In [6]:
for i in df.columns[1:]:
    IV(df, i, 'Def')

purpose_bin :  0.063
credit_policy_bin :  0.159
int_rate_bin :  0.235
log_annual_inc_bin :  0.02
installment_bin :  0.028
fico_bin :  0.178
days_with_cr_line_bin :  0.017
revol_bal_bin :  0.014
revol_util_bin :  0.013
inq_last_6mths_bin :  0.16
delinq_2yrs_bin :  0.002
pub_rec_bin :  0.022


In [7]:
df = df.drop(['delinq_2yrs_bin','int_rate_bin'], axis=1)
for i in df.columns[1:]:
    IV(df, i, 'Def')

purpose_bin :  0.063
credit_policy_bin :  0.159
log_annual_inc_bin :  0.02
installment_bin :  0.028
fico_bin :  0.178
days_with_cr_line_bin :  0.017
revol_bal_bin :  0.014
revol_util_bin :  0.013
inq_last_6mths_bin :  0.16
pub_rec_bin :  0.022


# 5. VIF

In [8]:
x = df.drop('Def', axis=1)
VIF(x)

const                    411.144605
purpose_bin                1.017684
credit_policy_bin          2.188749
log_annual_inc_bin         1.062156
installment_bin            1.024929
fico_bin                   1.300769
days_with_cr_line_bin      1.218018
revol_bal_bin              1.138896
revol_util_bin             1.018172
inq_last_6mths_bin         1.563400
pub_rec_bin                1.022287
dtype: float64
